In [ ]:
#LD morning phase quantification

import numpy as np
from scipy.signal import savgol_filter, find_peaks
import pandas as pd
import matplotlib.pyplot as plt

#input files
#locomotor_input = '/Users/donglinhan/Desktop/New New MozzieBox analysis/1min_by_Genotype/Line1Het_LD_only_1min.xlsx'
#locomotor_input = '/Users/donglinhan/Desktop/New New MozzieBox analysis/1min_by_Genotype/Line1Hom_LD_only_1min.xlsx'
#locomotor_input = '/Users/donglinhan/Desktop/New New MozzieBox analysis/1min_by_Genotype/WT_LD_only_1min.xlsx'

locomotor_data = pd.read_excel(locomotor_input)
total_n = 40 # change this according to number of columns
starting_column = 1
end_column = total_n + 1
all_peaks = pd.DataFrame()
window_start = 2760 # ZT22 of LD2
window_end = 3300 # ZT7 of LD3
peak_list = []
for i in range(starting_column, end_column):
    raw_data = locomotor_data.iloc[window_start:window_end, i]
    smoothed_data = savgol_filter(raw_data, window_length=361, polyorder=3)
    peaks, _ = find_peaks(smoothed_data)
    sorted_peaks = sorted(peaks, key=lambda p: smoothed_data[p], reverse=True)
    if sorted_peaks:
        highest_peak_index = sorted_peaks[0]
    if not sorted_peaks: 
        highest_peak_index = pd.NA
    if raw_data.max() < 5 or raw_data.sum() < 100:
        highest_peak_index = pd.NA
    ZT_time = highest_peak_index * (1/60) - 2
    peak_list.append(ZT_time)
all_peaks[locomotor_input] = pd.Series(peak_list)

#print(all_peaks)
#all_peaks.to_excel('Hom_morning_peaks.xlsx', index=False)
#all_peaks.to_excel('Het_morning_peaks.xlsx', index=False)
#all_peaks.to_excel('WT_morning_peaks.xlsx', index=False)

In [ ]:
#DD phase quantification

import numpy as np
from scipy.signal import savgol_filter, find_peaks
import pandas as pd
import matplotlib.pyplot as plt

#input files
#locomotor_input = '/Users/donglinhan/Desktop/New New MozzieBox analysis/phaseR/line1hom_LDDD_1_n=24.xlsx'
#locomotor_input = '/Users/donglinhan/Desktop/New New MozzieBox analysis/phaseR/line1het_LDDD_1_n=32.xlsx'
#locomotor_input = '/Users/donglinhan/Desktop/New New MozzieBox analysis/phaseR/line1het_LDDD_2_n=15.xlsx'
#locomotor_input = '/Users/donglinhan/Desktop/New New MozzieBox analysis/phaseR/WT_LDDD_1_n=32.xlsx'
#locomotor_input = '/Users/donglinhan/Desktop/New New MozzieBox analysis/phaseR/WT_LDDD_2_n=8.xlsx'

locomotor_data = pd.read_excel(locomotor_input)
valid_n = 24 # change this according to number of valid animals 
starting_column = 10 
end_column = starting_column + valid_n
all_peaks = pd.DataFrame()

for d in range(0, 7): 
    DD_start = 4320 + (d * 1440)
    DD_end = 5759 + (d * 1440)
    peak_list = []

    for i in range(starting_column, end_column):
        raw_data = locomotor_data.iloc[DD_start:DD_end, i]
        smoothed_data = savgol_filter(raw_data, window_length=361, polyorder=3)
        peaks, _ = find_peaks(smoothed_data)
        sorted_peaks = sorted(peaks, key=lambda p: smoothed_data[p], reverse=True)
        if sorted_peaks:
            highest_peak_index = sorted_peaks[0]
        if not sorted_peaks: 
            highest_peak_index = pd.NA
        #highest_peak_index = np.argmax(smoothed_data) 
        if raw_data.max() < 5 or raw_data.sum() < 100:
            highest_peak_index = pd.NA
        CT_time = highest_peak_index * (1/60)
        peak_list.append(CT_time)
    all_peaks[d] = pd.Series(peak_list)

#print(all_peaks)
#all_peaks.to_excel('Hom_1_24_DDpeaks.xlsx', index=False)
#all_peaks.to_excel('Het_1_32_DDpeaks.xlsx', index=False)
#all_peaks.to_excel('Het_2_15_DDpeaks.xlsx', index=False)
#all_peaks.to_excel('WT_1_32_DDpeaks.xlsx', index=False)
#all_peaks.to_excel('WT_2_8_DDpeaks.xlsx', index=False)

In [ ]:
## Plotting raose plot/circular plot and perform necessary stats

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import circmean, rayleigh
from pingouin import circ_rayleigh, circ_r, circ_mean, plot_circmean
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd

peak_input = '/Users/donglinhan/Desktop/New New MozzieBox analysis/All_DDpeaks.xlsx'
all_peaks = pd.read_excel(peak_input)

# Create rose plot
plot_width = 3
plot_height = 3
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(3*plot_width, 1*plot_height), subplot_kw=dict(projection='polar'))
bin_start = np.radians (0)
bin_end = np.radians (360)
#pdf_pages = PdfPages('DD1_rose_plot.pdf')
#pdf_pages = PdfPages('DD2_rose_plot.pdf')
#pdf_pages = PdfPages('DD3_rose_plot.pdf')

#DD1
ax = axes [0]
WT_time_values_raw = all_peaks.iloc[:, 2]
WT_time_values = WT_time_values_raw.dropna()
WT_radians = np.radians(np.array(WT_time_values.to_list()) * 360 / 24)
ax.scatter(WT_radians, np.ones_like(WT_radians) * 0.9, color='#7f7f7f',alpha = 1, linewidth = 0, s=10)
mean_direction = - circ_mean(WT_radians) + (np.pi/2)
vector_length = circ_r(WT_radians)
ax.quiver(0, 0, np.cos(mean_direction)*vector_length, np.sin(mean_direction)*vector_length, scale=2, color='#7f7f7f', width=0.01)
ax.set_theta_zero_location('N')
ax.set_rticks([])
ax.set_theta_direction(-1)
# Set labels
hours = []
ax.set_xticks(np.arange(0, 2*np.pi, 2*np.pi/4))
ax.set_xticklabels(hours)

#Het
ax = axes [1]
time_values_raw = all_peaks.iloc[:, 9]
time_values = time_values_raw.dropna()
Het_radians = np.radians(np.array(time_values.to_list()) * 360 / 24)
#ax.hist(Het_radians, bins=24, range = (bin_start, bin_end), color='#c88383', density=True)
ax.scatter(Het_radians, np.ones_like(Het_radians) * 0.9, color='#c88383', alpha = 1, linewidth = 0, s=10)
mean_direction = - circ_mean(Het_radians) + (np.pi/2)
vector_length = circ_r(Het_radians)
ax.quiver(0, 0, np.cos(mean_direction)*vector_length, np.sin(mean_direction)*vector_length, scale=2, color='#c88383', width=0.01)
ax.set_theta_zero_location('N')
ax.set_rticks([])
ax.set_theta_direction(-1)
hours = []
ax.set_xticks(np.arange(0, 2*np.pi, 2*np.pi/4))
ax.set_xticklabels(hours)


#Het
ax = axes [2]
time_values_raw = all_peaks.iloc[:, 16]
time_values = time_values_raw.dropna()
Hom_radians = np.radians(np.array(time_values.to_list()) * 360 / 24)
#ax.hist(Hom_radians, bins=24, range = (bin_start, bin_end), color='#d04546', density=True)
ax.scatter(Hom_radians, np.ones_like(Hom_radians) * 0.9, color='#d04546', alpha = 1, linewidth = 0, s=10)
mean_direction = - circ_mean(Hom_radians) + (np.pi/2)
vector_length = circ_r(Hom_radians)
ax.quiver(0, 0, np.cos(mean_direction)*vector_length, np.sin(mean_direction)*vector_length, scale=2, color='#d04546', width=0.01)

# Set zero angle to the top
ax.set_theta_zero_location('N')
ax.set_rticks([])
# Set clockwise direction
ax.set_theta_direction(-1)
# Set labels
hours = []
ax.set_xticks(np.arange(0, 2*np.pi, 2*np.pi/4))
ax.set_xticklabels(hours)

plt.show()

pdf_pages.savefig(fig, bbox_inches='tight')
pdf_pages.close()

z, pval = circ_rayleigh(WT_radians)
r = z / len(WT_radians)
print (r, pval)

z, pval = circ_rayleigh(Het_radians)
r = z / len(Het_radians)
print (r, pval)

z, pval = circ_rayleigh(Hom_radians, d = 2*np.pi/24)
r = z / len(Hom_radians)
print (r, pval)